In [16]:
import openai
from dotenv import load_dotenv
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

load_dotenv()

from custom_gmail_reader import CustomGmailReader

# Instantiate the CustomGmailReader
loader = CustomGmailReader(
    query="",
    max_results=50,
    results_per_page=10,
    service=None
)

# Load the emails
documents = loader.load_data()


index = VectorStoreIndex.from_documents(documents)

# Create a retriever to fetch relevant documents
retriever = index.as_retriever(retrieval_mode='similarity', k=3)

prompt = "Here is a list of emails regarding the topic you will see at the bottom of this question: \n"

# Define your query
query = "What did Olivia email me about?"

# Retrieve relevant documents
relevant_docs = retriever.retrieve(query)

print(f"Number of relevant documents: {len(relevant_docs)}")
print("\n" + "="*50 + "\n")

# for i, doc in enumerate(relevant_docs):
#     print(f"Document {i+1}:")
#     print(f"Text sample: {doc.node.get_content()[:200]}...")  # Print first 200 characters
#     print(f"Metadata: {doc.node.metadata}")
#     print(f"Score: {doc.score}")
#     print("\n" + "="*50 + "\n")

for i, doc in enumerate(relevant_docs):
    prompt += f"Document {i+1}:"
    prompt += f"Text sample: {doc.node.get_content()}..." 
    prompt += f"Metadata: {doc.node.metadata}"
    prompt += f"Score: {doc.score}"
    # print("\n" + "="*50 + "\n")



Number of relevant documents: 2




In [26]:
from langfuse.decorators import observe
from llama_index.core.query_engine import RetrieverQueryEngine


query_engine = RetrieverQueryEngine(retriever=retriever)

@observe
def ask_away(s, prompt):
    ques = prompt + "\n" + "Question is: " + s
    response = query_engine.query(ques)
    print(response, '\n')
    

In [27]:
# print(prompt)
ask_away("What did Olivia email me about?", prompt)

Olivia emailed you to confirm the scheduled call for 130 PT on Wednesday and provided information on how to best prepare for the interview, including reviewing the company website, watching culture and product videos, and being ready to discuss instances of exceptional performance or notable results. Additionally, she mentioned the Shield AI 'Benefits at a Glance' and asked for confirmation if the scheduled time no longer works. 



In [29]:
ask_away("Did I respond correctly? Did I sound too nervous or eager or desperate?", prompt)

You responded appropriately and did not sound nervous, eager, or desperate in your email exchanges. 

